<img src="https://oasislmf.org/packages/oasis_theme_package/themes/oasis_theme/assets/src/oasis-lmf-colour.png" alt="Oasis LMF logo" width="250"/>

# Excercise 1:  Introduction to the Oasis Model Development Kit (MDK).

The MDK is a Python based toolkit that allows Oasis models to be executed locally, using the same data formats and execution components as in the Oasis platform. The MDK is available as a Python package on PYPI. Currently the MDK only supports ground up loss calculations, and insured loss calculations from the Oasis platform will be integrated in the coming monthes.

The main goal of the MDK is to streamline the model development and test process. This can all be ran on an analysis server, with no need to access a deplyed Oasis platform. However, having full modelling capabilities available programatically provides some interesting potential for advanced analytics and data visualization, especially when combined with a modern data science toolkit.

The first version of the MDK was released in early 2018 and it has been used on several model development projects. We are keen to get feedback on usability and ideas for other useful features. 

## Excercise goals
* Understand the Oasis analysis flow
* Understand the functionality provided by the "oasislmf model" sub-commands
* Understand the configuration options of the "oasislmf model" sub-commands

## Oasis Analysis Flow

<img src="images/mdk_flow.png" alt="MDK" width="100%"/>

The modelling capabilities of the MDK are exposed by the "oasislmf model" sub-commands. The list of commands are:

#### oasislmf model generate-oasis-files
Test the generation of the analysis input files for an exposure set.
#### oasislmf model generate-losses*
Test the generation of ground up losses and metrics for a set of analysis input files.
#### oasislmf model run
Test the generation of ground up losses and metrics for an exposure set. This essentially chains the other commands together.

The inputs/ouputs are:

#### A
* source_exposures_validation_file_path
 	File path to schema for validating source exposure, in XSD format.
* source_to_canonical_exposures_transformation_file_path
 	File path to transform to select and store a subset of the source exposure, in XSLT format.
* canonical_exposures_profile_json_path
 	File path to profile that describes the structure of the stored data.
* canonical_exposures_validation_file_path
 	File path to schema for validating stored exposure, in XSD format.
* canonical_to_model_exposures_transformation_file_path
 	File path to transform to select a subset of the stored exposure for invoking the keys lookup logic, in XSD format.
#### B
* model_exposures_file_path
    File path to the inputs to the model keys lookup class in CSV format.
#### C
* keys_data_path
    Path to directory containing data files required by the model keys lookup class.
#### D
* keys_file_path
    Path for the file containing keys for locations with successful lookups.
#### E
* keys_error_file_path
    Path for the file containing keys for locations with failing or non-matching looups.
#### F
* oasis-files-path 
    Path to Oasis files, the analysis inputs in ktools format.
#### G
* model_data_path
    Path to directory containing the model data.
    
Note there are additional commands for more granular functionality and data manipulation that we will not cover today.

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
import json

In [ ]:
! oasislmf version

In [ ]:
! oasislmf model --help

## generate-oasis-files command

In [ ]:
! oasislmf model generate-oasis-files --help

In [ ]:
! oasislmf model generate-oasis-files \
    -o "oasis_files" \
    -k "keys_data/EuWs" \
    -v "keys_data/EuWs/ModelVersion.csv" \
    -p "wisc-euws-canonical-loc-profile.json" \
    -e "source_data/SourceLocWISC_Paris.csv" \
    -a "validation_files/Generic_Windstorm_CanLoc_A.xsd" \
    -b "transformation_files/MappingMapToGeneric_Windstorm_CanLoc_A.xslt" \
    -c "validation_files/WiscModelLoc.xsd" \
    -d "transformation_files/MappingMapToModelLoc_WISC_3.xslt"


## generate-losses command

In [ ]:
! oasislmf model generate-losses --help

In [ ]:
! rm -rf /tmp/run
! oasislmf model generate-losses \
    -o "oasis_files" \
    -j "analysis_settings.json" \
    -m "model_data/EuWs" \
    -r "/tmp/run"

## run command

In [ ]:
! oasislmf model run --help

In [ ]:
! rm -rf /tmp/run
! oasislmf model run \
    -k "keys_data/EuWs" \
    -l "src/keys_server" \
    -p "wisc-euws-canonical-loc-profile.json" \
    -e "source_data/SourceLocWISC_Paris.csv" \
    -a "validation_files/Generic_Windstorm_CanLoc_A.xsd" \
    -b "transformation_files/MappingMapToGeneric_Windstorm_CanLoc_A.xslt" \
    -c "validation_files/WiscModelLoc.xsd" \
    -d "transformation_files/MappingMapToModelLoc_WISC_3.xslt" \
    -j "analysis_settings.json" \
    -m "model_data/EuWs" \
    -r "/tmp/run"

## Using the oasislmf.json model configuration file

In [ ]:
with open('./oasislmf.json', 'r') as myfile:
    analysis_settings=json.loads(myfile.read().replace('\n', ''))
print(json.dumps(analysis_settings, indent=True))

In [ ]:
! rm -rf /tmp/analysis_test
! oasislmf model run -e "tests/data/SourceLocPiWind10K.csv" -r /tmp/analysis_test